In [3]:
# !conda install -c conda-forge python-igraph

In [1]:
from igraph import Graph as IGraph
import pandas as pd
from py2neo import Node, Relationship, Graph
import numpy as np

In [2]:
df = pd.read_csv('/home/ubuntu/Scarlett/Input/ED_main_cuis_input.csv')

In [3]:
df.head()

,No.,cuis,sc.name,icd9,icd9_code
0,1,C0859643,Senile dementia with delusional or depressive ...,2902.0,'2902'
1,2,C0154315,Senile dementia with delirium,2903.0,'2903'
2,3,C0154319,Other specified senile psychotic conditions,2908.0,'2908'
3,4,C1457889,Unspecified senile psychotic condition,2909.0,'2909'
4,5,C0001957,Alcohol withdrawal delirium,2910.0,'2910'


In [4]:
# df['cuis']

In [5]:
graph = Graph("http://10.200.112.233:7474/db/data/", user="scarlett", password="Admin123")

In [6]:
def get_children_nodes(concept_id):
    query = '''
    MATCH (c:Concepts)<-[r:cui_2_cui{relation:'isa'}]-(child:Concepts)
    where c.id = \"'''+ concept_id + '''\"
    return c.id, c.name, child.id, child.name
    '''
    data = graph.run(query).to_data_frame()
    return data


In [7]:
# get_children_nodes('C0233407')

In [8]:
visited, queue = set(), []
tuples = []
dfs = []

for cid in set(df['cuis']):
    visited.add(cid)
    queue.append(cid)
    
while queue:
    s = queue.pop(0)
    
    children = get_children_nodes(s)
    if len(children) > 0:
        for child in list(children['child.id']):
            if child not in visited:
                visited.add(child)
                queue.append(child)

            tuples.append([s, child])
        dfs.append(children)

In [9]:
concepts = pd.concat(dfs)
concepts = {**dict(zip(concepts['c.id'], concepts['c.name'])), **dict(zip(concepts['child.id'], concepts['child.name']))}

In [10]:
# tuples

In [11]:
ig = IGraph.TupleList(tuples, weights=False, directed=True)

In [12]:
ig.summary()

'IGRAPH DN-- 1755 2431 -- \n+ attr: name (v)'

In [13]:
# layout = ig.layout("kk")
# plot(ig, layout = layout)

In [14]:
# ig.write_graphml('data/children.GraphML')

In [15]:
def net_stat(g):
    node_n = g.vcount()
    edge_m = g.ecount()
    degree_mean = np.mean(g.indegree())
    degree_std = np.std(g.indegree())
    outdegree_mean = np.mean(g.outdegree())
    outdegree_std = np.std(g.outdegree())
    density = g.density()
#     avg_path = g.average_path_length(directed=True, unconn=True)
    avg_path = 0
    components = g.clusters(mode='WEAK')
    comp_count = len(components)
    giant_comp = components.giant()
    giant_comp_r = float(giant_comp.vcount())/node_n
    cluster_co_global = g.transitivity_undirected()
    # cluster_co_avg = g.transitivity_avglocal_undirected()
    recip = g.reciprocity()
    assort = g.assortativity_degree(directed=True)
    print ('#Node, #Edge, <kin>, std(kin), <kout>, std(out), #Density, #Path, #Comp, %GCR, Cluster, Recip, Assort')
    print ('%d, %d, %.3f, %.3f, %.3f, %.3f,%.3f, %.3f, %d, %.3f, %.3f, %.3f, %.3f ' % (node_n, edge_m, degree_mean, degree_std, outdegree_mean, outdegree_std, density, avg_path, comp_count, giant_comp_r, cluster_co_global, recip, assort))


In [16]:
net_stat(ig)

#Node, #Edge, <kin>, std(kin), <kout>, std(out), #Density, #Path, #Comp, %GCR, Cluster, Recip, Assort
1755, 2431, 1.385, 0.632, 1.385, 4.054,0.001, 0.000, 10, 0.916, 0.044, 0.003, -0.186 


### There are 10 components

In [17]:
ig.vs['ind'] = ig.indegree()

### Root nodes 

In [19]:
for n in ig.vs.select(ind=0)['name']:
    print(n, concepts[n])

C0018524 Hallucinations
C0278166 Glasgow coma scale, 5
C0029226 Hallucinations, Organic
C0009676 Confusion
C0278165 Glasgow coma scale, 4
C0085584 Encephalopathies
C0859643 senile dementia with delusional or depressive features
C0278167 Glasgow coma scale, 6
C1821572 Decreased level of consciousness
C0237284 unresponsive behavior
C0152129 Pathological drug intoxication
C0154315 Senile dementia with delirium
C0233522 Inappropriate behavior
C0278061 Abnormal mental state
C0233407 Disorientation
C0278164 Glasgow coma scale, 3
C0033975 Psychotic Disorders
C0085631 Agitation
C0011253 Delusions
C0023380 Lethargy
C0154309 Presenile dementia with delirium
C0009421 Comatose
